### Imports

In [52]:
import numpy as np
import pandas as pd
from pandas import read_excel
from csv import reader
import gurobipy as grb
from gurobipy import *

### Station

In [53]:
'''
Station Object
- Pickup / Delivery Nodes
- Charging Nodes
'''
class Station():
    
    def __init__(self,nodeId,name, stType, mhTime):
        self.nodeId = int(nodeId) # an integer representing the unique id of a node
        self.name = name #name of the node
        self.stType = stType #station Type (P/D/C/PD)
        self.mhTime = mhTime # material handling time associated with the station 
    
    def __repr__(self):
        return f"Station#{self.nodeId} - {self.name} - {self.stType}"
    
    def getType(self):
        '''
        Returns type of node
        '''
        return self.stType
    
    def getNode(self):
        '''
        returns integer id of the node
        '''
        return self.nodeId
    
    def getMHT(self):
        '''
        returns material handling time at the station
        '''
        return self.mhTime

### Task
This class represents the following types of tasks:
Charge Task,
Non-Critical Charge task,
MH Task,
Unloaded Travel Task

In [54]:
'''
Class representing a task
Charge Task
Non-Critical Charge task
MH Task
Unloaded Travel Task
'''
class Task():
    
    def __init__(self, taskId, taskType, source, dest):
        self.taskId=taskId #this is the original id of the request from the file
        self.taskIndex=888 #this is changed based on order of tasks
        self.taskType = taskType
        self.source = source
        self.dest = dest
        
    def __repr__(self):
        return f"{self.taskId}, type:{self.taskType}, from:{self.source}, to:{self.dest}"

### Transport Order Task (Inherited from Task)

In [55]:
'''
This class represents an inherited object(Transport Order) of a TASK'''
class TransportOrder(Task):
    def __init__(self,taskId, taskType, source, dest, ept, ldt, cap):
        super().__init__(taskId, taskType, source, dest) #call 'Task' class constructor
        self.ept = ept
        self.ldt = ldt
        self.cap = cap
        
    def __repr__(self):
        return f"{self.taskId}, type:{self.taskType}, from:{self.source}, to:{self.dest}, ept:{self.ept}, ldt:{self.ldt}, cap:{self.cap}"

### Layout

In [56]:
class Layout():
    
    def __init__(self, fileName):
        self.distMat=[]
        self.readDistanceMatrix(fileName)
    
    def readDistanceMatrix(self,fileName):
        dm = reader(open(fileName))
        self.distMat = list(dm)
        self.distMat[0][0]=0
    
    def getDistanceFromNode(self, source, dest):
        '''
        returns the distance (in m) between 2 nodes
        '''
        return float(self.distMat[source][dest])

### AGV

In [57]:
class AGV():
    def __init__(self, agvId, startNode, caps, speed, charge, dischargeRate, chargeRate, taskList, travelCost, low=30, up=60):
        
        self.agvId = agvId #agv id
        self.startNode = startNode #initial node
        self.caps = caps #capability
        self.speed = speed # speed in m/s
        self.charge = charge # charge %
        self.dischargeRate = dischargeRate # % per second
        self.chargeRate = chargeRate # % per second
        self.taskList = list() # list of tasks
        self.release=(0, startNode) #(time, location)
        self.travelCost = travelCost # weighted travel cost
        self.LOWER_THRESHOLD = low # lower threshold of charging
        self.UPPER_THRESHOLD =up # upper threshold of charging
        self.state = 'N'
    
    def __repr__(self):
        return f"AGV{self.agvId},Charge-{self.charge}%,TCost-{self.travelCost}"
    
    def getSpeed(self):
        '''
        Returns speed of agv in m/s
        '''
        return self.speed
    
    def getChargeRate(self):
        '''
        Returns charging rate of agv in %/sec
        '''
        return self.chargeRate
    
    def getDischargeRate(self):
        '''
        Returns discharging rate of agv in %/sec
        '''
        return self.dischargeRate
    
    def getCurrentReleaseNode(self):
        '''
        returns the current node of AGV
        '''
        return self.release[1]
    
    def getCurrentReleaseTime(self):
        '''
        returns the time at which AGV leaves/can leave the release node
        '''
        return self.release[0]
    
    def setCurrentReleaseNode(self,releaseNode):
        '''
        updates the node of AGV
        '''
        self.release=(self.release[0],releaseNode)
        
    def setCurrentReleaseTime(self,releaseTime):
        '''
        updates the node of AGV
        '''
        self.release=(releaseTime,self.release[1])
        
    def setState(self,state):
        '''
        Sets the state of agv to Normal ='N' or Charging = 'C'
        '''
        self.state = state
        
    def getState(self):
        '''
        Returns state of agv, 'N'=Normal, 'C'=Charging
        '''
        return self.state
        
    def setCharge(self, charge):
        '''
        Sets the agv charge
        '''
        self.charge = charge
        
    def getCharge(self):
        '''
        Returns AGV Charge in %age
        '''
        return self.charge
    
    def getStartNode(self):
        '''
        Returns agv's initial/start node'''
        return self.startNode
    
    

### Scheduler Class
This class is responsible for scheduling tasks on AGVs based on system state

In [58]:
class Scheduler():
    
    
    def __init__(self, layoutFile, agvFile, requestFile, stationFile):
        self.agvs =[] # field responsible to keep track of agvs
        self.stations=[] # field responsible to keep track of stations
        self.layout = None 
        self.transportOrders = list()
        self.chargingStations = list()
        self.taskList = dict() # this contains a processed list of TOs and Chargetasks, produced during greedy sequencing
        self.taskSequence = dict() # it is made AFTER all tos and charge tasks are assigned i.e. after greedy sequencing, taskSequence is sent for scheduling
        self.taskSchedule = dict() # order of execution -> create taskList->create taskSequence->schedule via LP->taskSchedule
        self.createLayout(layoutFile=layoutFile)
        self.createAGVs(agvFile=agvFile)
        self.createStations(stationFile=stationFile)
        self.createRequests(requestFile=requestFile)
      
    def schedulerDebug(self, agv=None, task=None, dtype=0):
        if dtype==0:
            print(f"agv info:{agv}, release info:{agv.release}")
        elif dtype==1:
            print(f"task info:{task}")
        
    def createLayout(self, layoutFile):
        '''
        Creates digital layout inside scheduler
        '''
        self.layout = Layout(fileName=layoutFile)
        
    def createAGVs(self, agvFile):
        '''
        Creates agvs
        '''
        df =read_excel(agvFile)
        #TODO - modify lower threshold of agv based on 2*max distance in the layout and discharge rate of agvs
        for index,row in df.iterrows():
            agv = AGV(agvId=row['agvidf'], startNode=row['startNode'],caps= row['capability'], speed=row['speed'], \
                      charge=row['charge'],dischargeRate= row['dischargeRate'], chargeRate = row['chargingRate'],\
                      travelCost = row['travelCost'], low = row['lowerThreshold'], up = row['upperThreshold'], taskList=None)
            self.agvs.append(agv)

    def createStations(self, stationFile):
        '''
        Creates stations
        '''
        df = read_excel(stationFile)
        for index, row in df.iterrows():
            station = Station(nodeId=row['id'], name=row['pointidf'], stType=row['type'], mhTime=row['mhtime'])
            self.stations.append(station)
            if station.getType() == 'C':
                self.chargingStations.append(station)
        
    
    def createRequests(self, requestFile):
        '''
        Creates TOs that need to be scheduled
        '''
        df =read_excel(requestFile)
        for index,row in df.iterrows():
            transportOrder = TransportOrder(taskId=row['Id'], taskType='TO', source=row['source'],\
                                            dest= row['target'],ept= row['ept'], ldt=row['ldt'], cap=row['capability'])
            self.transportOrders.append(transportOrder)

            
#'''
#HEREUNDER LIES THE GREEDY SEQUENCING HEURISTIC
#'''

    def createGreedySequence(self):
        '''
        This function creates a greedy sequence of tasks to be performed, calls createTaskSequence to make a sequence list 
        with unloaded travel etc.
        '''
        self.transportOrders.sort(key = lambda x: x.ldt) # sort based on delivery time
        #assign tasks to AGVs and keep checking for expected charge after finishing the task
        for to in self.transportOrders:
            print(f'TO to schedule:{to}')
            agv_count=[agv for agv in self.agvs if to.cap in agv.caps]
            print(f"No. of capable agvs:{len(agv_count)}")
            # assert that there is at least one agv with desired capability
            assert (len(agv_count)>0), ("There should always be at least one agv with desired capability")
            if len(agv_count)==1:
                #add the task to AGV
                print(f"1-agv selected:{agv_count[0]}")
                self.addGreedyTask(agv_count[0],to)
            elif len(agv_count)>1:
                scores=[]
                for agv in agv_count:
                    score = self.getScore(agv,to)
                    scores.append(score)
                optAGV = scores.index(min(scores))
                print(f"n-agv selected:{agv_count[optAGV]}")
                self.addGreedyTask(agv_count[optAGV],to)
    
    
    def getScore(self,agv,task):
        srcStation = list(filter(lambda x:x.nodeId==task.source,self.stations))[0] # src station
        dstStation = list(filter(lambda x:x.nodeId==task.dest,self.stations))[0] # destination station
        if agv.getCharge()<agv.LOWER_THRESHOLD:
            #unloaded travel Cost
            dists = [self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),station.getNode()) for station in self.chargingStations]
            optIndex = dists.index(min(dists))
            nearestChargeNode = self.chargingStations[optIndex].getNode()
            distScore = (self.layout.getDistanceFromNode(nearestChargeNode,task.source)+self.layout.getDistanceFromNode(task.source,task.dest))*agv.travelCost
            
            #tardiness score
            tardScore=0
            travelDist = self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),nearestChargeNode)
            drivingTime = travelDist/agv.speed # time spent in driving
            minChargeTime = (agv.LOWER_THRESHOLD - agv.getCharge())/agv.chargeRate
            absRelTime = agv.getCurrentReleaseTime()+minChargeTime+drivingTime
            dist = self.layout.getDistanceFromNode(nearestChargeNode,task.source)+\
                self.layout.getDistanceFromNode(task.source,task.dest)
            
            if absRelTime>task.ept:
                time = absRelTime+dist/agv.speed +srcStation.mhTime + dstStation.mhTime
            else:
                time = task.ept+dist/agv.speed +srcStation.mhTime + dstStation.mhTime
            
            tardiness = max(0,time-task.ldt)
            
            tardScore = tardiness**2
            
            return tardScore + distScore
        
        else:
            #dist score
            distScore = (self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),task.source)+self.layout.getDistanceFromNode(task.source,task.dest))*agv.travelCost
            #tardiness score
            tardScore=0
            dist = self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),task.source)+\
                self.layout.getDistanceFromNode(task.source,task.dest)
            
            absRelTime = max(agv.getCurrentReleaseTime(),task.ept)
            
            time = absRelTime+dist/agv.speed+srcStation.mhTime+dstStation.mhTime
            
            tardiness = max(0,time-task.ldt)
            
            tardScore = tardiness**2
            
            return tardScore + distScore
        
    
    def addChargeTask(self,agv):
        '''
        Adds a charge task to the agv's tasklist
        '''
        dists = [self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),station.getNode()) for station in self.chargingStations]
        optIndex = dists.index(min(dists))
        nearestChargeNode = self.chargingStations[optIndex].getNode()
        chargeTask = Task(999,'C',agv.getCurrentReleaseNode(),nearestChargeNode)
        
        agv.taskList.append(chargeTask) 
        agv.setState('C') #charging
        
        travelDist = self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),nearestChargeNode)
        drivingTime = travelDist/agv.speed # time spent in driving
        agv.setCharge( agv.charge - (agv.dischargeRate * drivingTime) )
        
        agv.setCurrentReleaseNode(nearestChargeNode)
        agv.setCurrentReleaseTime(agv.getCurrentReleaseTime()+drivingTime)
        self.schedulerDebug(dtype=0, agv=agv)
        self.schedulerDebug(dtype=1, task=task)

        
    def addTaskToTaskList(self,agv,task):
        #add a TO to agv's task list 
        travelDist = self.layout.getDistanceFromNode(agv.getCurrentReleaseNode(),task.source) + \
                        self.layout.getDistanceFromNode(task.source, task.dest)
            
        srcStation = list(filter(lambda x:x.nodeId==task.source,self.stations))[0] # src station
        dstStation = list(filter(lambda x:x.nodeId==task.dest,self.stations))[0] # destination station
        drivingTime = travelDist/agv.speed # time spent in driving
        
        if agv.getState()=='N':
            travelTime = drivingTime + srcStation.mhTime + dstStation.mhTime # total time including material handling
            agv.taskList.append(task)
            agv.setCurrentReleaseNode(task.dest)
            agv.setCurrentReleaseTime(max(agv.getCurrentReleaseTime(),task.ept)+travelTime)
            agv.setCharge( agv.charge - (agv.dischargeRate * drivingTime) )
            agv.setState('N')
            self.schedulerDebug(dtype=0, agv=agv)
            self.schedulerDebug(dtype=1, task=task)
        
        elif agv.getState()=='C':
            minChargeTime = (agv.LOWER_THRESHOLD - agv.getCharge())/agv.chargeRate # time to reach LOWER_THRESHOLD charge level
            minChargeAbsTime = agv.getCurrentReleaseTime() + minChargeTime  # the absolutime time (in sec) at which AGV becomes 30% charged
            
            if task.ept >= minChargeAbsTime :
                # add the task but update charge based on delta
                chargeTime = (task.ept - agv.getCurrentReleaseTime())
                agv.setCharge(agv.charge + (chargeTime * agv.chargeRate)) # charge after charging till task's ept
                agv.setCurrentReleaseTime(agv.getCurrentReleaseTime()+chargeTime)
                travelTime = drivingTime + srcStation.mhTime + dstStation.mhTime # total time including material handling
                agv.taskList.append(task)
                agv.setCurrentReleaseNode(task.dest)
                agv.setCurrentReleaseTime(max(agv.getCurrentReleaseTime(),task.ept)+travelTime)
                agv.setCharge( agv.charge - (agv.dischargeRate * drivingTime) )
                agv.setState('N')
                self.schedulerDebug(dtype=0, agv=agv)
                self.schedulerDebug(dtype=1, task=task)
                
            elif task.ept < minChargeAbsTime:
                #
                agv.setCurrentReleaseTime(agv.getCurrentReleaseTime()+minChargeAbsTime)
                travelTime = drivingTime + srcStation.mhTime + dstStation.mhTime # total time including material handling
                agv.taskList.append(task)
                agv.setCurrentReleaseNode(task.dest)
                agv.setCurrentReleaseTime(max(agv.getCurrentReleaseTime(),task.ept)+travelTime)
                agv.setCharge( agv.charge - (agv.dischargeRate * drivingTime) )
                agv.setState('N')
                self.schedulerDebug(dtype=0, agv=agv)
                self.schedulerDebug(dtype=1, task=task)
                
         
        
    def addGreedyTask(self,agv,task):
        if agv.charge>=agv.LOWER_THRESHOLD and agv.state =='N':
            #add the task
            self.addTaskToTaskList(agv,task)
            
        elif agv.charge<agv.LOWER_THRESHOLD and agv.state=='N':
            # add a charge task
            self.addChargeTask(agv)
            
        if agv.charge<agv.LOWER_THRESHOLD and agv.state=='C':
            #update the charge at time of new request, if sufficient charge is present, change state, add the task
            self.addTaskToTaskList(agv,task)
            
    
    def writeToTaskSequence(self, agvId, task):
        '''
        write a task of an agv to scheduler's task sequence 
        '''
        taskDict={}
        taskDict['taskIndex']=task.taskIndex
        taskDict['taskId']=task.taskId
        taskDict['taskType']=task.taskType
        taskDict['source']=task.source
        taskDict['dest']=task.dest
        taskDict['mhTime'] = list(filter(lambda x:x.nodeId==task.source,self.stations))[0].getMHT()
        if task.taskType=='TO':
            taskDict['ept']=task.ept
            taskDict['ldt']=task.ldt
        self.taskSequence.get(agvId).append(taskDict)
    
    def createTaskSequence(self):
        self.taskSequence={}
        #create keys for AGVs
        for agv in self.agvs:
            for task in agv.taskList:
                task.taskIndex = agv.taskList.index(task) #assigning index
        #create agv keys
        for agv in self.agvs:
            self.taskSequence[agv.agvId]=[]
            ut = Task(998,'UT',agv.getStartNode(), agv.taskList[0].source)#create a UT(Unloaded Travel) task, id and index dont matter
            self.writeToTaskSequence(agv.agvId,ut)
            for t,task in enumerate(agv.taskList):
                self.writeToTaskSequence(agv.agvId,task)
                if t<len(agv.taskList)-1 and agv.taskList[t+1].taskType!='C':
                    ut = Task(998,'UT',task.dest, agv.taskList[t+1].source)
                    self.writeToTaskSequence(agvId=agv.agvId,task=ut)
                                
#'''
#HEREUNDER LIES THE LP FORMULATION / could be another class
#'''   
    
    
    def solveLP(self,printOutput=False):
        numAGVs = len(self.agvs)
        REQ = []
        SRC=[]
        DEST=[]
        EPT=[]
        LDT=[]
        MHT=[]
        cr=[]
        dcr=[]
        sp=[]
        REQID=[]
        
        #keep track of tasks (UT, TO , C) in agvs
        for a,agv in enumerate(self.agvs):
            REQ.append(len(self.taskSequence.get(a)))
            sp.append(agv.getSpeed())
            cr.append(agv.getChargeRate())
            dcr.append(agv.getDischargeRate())
            SRC.append([])
            DEST.append([])
            EPT.append([])
            LDT.append([])
            MHT.append([])
            REQID.append([])
            
            for r in range(REQ[a]):
                REQID[a].append(self.taskSequence.get(a)[r]['taskId'])
                SRC[a].append(self.taskSequence.get(a)[r]['source'])
                DEST[a].append(self.taskSequence.get(a)[r]['dest'])
                MHT[a].append(self.taskSequence.get(a)[r]['mhTime'])
                EPT[a].append(self.taskSequence.get(a)[r].get('ept') or 0)
                LDT[a].append(self.taskSequence.get(a)[r].get('ldt') or 0)
                
        
        C = [x.getNode() for x in self.chargingStations] # list of charging station nodeIDs
        
        
        #GUROBI
        m = grb.Model('Scheduling')
        m.setParam('OutputFlag',printOutput)
        #minimization model
        m.modelSense = GRB.MINIMIZE
        #decision variables
        #variable for lateness of request r
        Z = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"Z_{r}_{v}")
             for v in range(numAGVs)  for r in range(REQ[v])} # 0 becuase we do not consider earliness

        #variable to represent the time instance at which AGV reaches 'source' of a request r
        S = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"S_{r}_{v}")
             for v in range(numAGVs)  for r in range(REQ[v])}

        #variable to represent the time instance at which AGV reaches 'destination' of a request r
        D = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = f"D_{r}_{v}") 
             for v in range(numAGVs)  for r in range(REQ[v])}

        #variable to represent the battery status at the beginning of request r
        B = {(r,v):m.addVar(vtype = GRB.CONTINUOUS, lb = 30.0, ub=100.0, name = f"B_{r}_{v}")
             for v in range(numAGVs)  for r in range(REQ[v])}

        #battery can reach 0% only at charging stations
        for v in range(numAGVs):
            for r in range(REQ[v]):
                if SRC[v][r] in C:
                    B[r,v].lb=0.0 # gurobi syntax to showcase indices
                    
        #objective function
        #Objective 1 - To minimize lateness
        obj_lateness = grb.quicksum(Z[r,v]*Z[r,v] for r in range(REQ[v]) for v in range(numAGVs))

        # #Objective 2 - To maximize charging duration and minimize parking duration at source and destination nodes
        obj_charging = grb.quicksum(0 if SRC[v][r] in C else 1*(S[r+1,v]-S[r,v]) for r in range(REQ[v]-1) for v in range(numAGVs))

        #Objective 3 - To minimize Unloaded Travel Time
        obj_unloaded = grb.quicksum(0 if REQID[v][r]<998 else 1*(D[r,v] - S[r,v]) for r in range(REQ[v]) for v in range(numAGVs)) 
        # becoz that's where unloaded travel occurs
        
        #SET OBJECTIVE
        m.setObjective(obj_lateness+obj_charging)
        
        #Constraints

        #Constraint 1 - trip time should be less than Destination time instance
        for v in range(numAGVs):
            for r in range(REQ[v]):
                i=SRC[v][r]
                j=DEST[v][r]
                dist = self.layout.getDistanceFromNode(i,j)
                m.addConstr(S[r,v]+(MHT[v][r]+(1/sp[v])*dist)<=D[r,v],name=f"Headway_{r}_{v}")
                
        #Constraint 2 - Destination of request should equal source of next request
        for v in range(numAGVs):
            for r in range(REQ[v]-1):
                i=SRC[v][r]
                j=DEST[v][r]
                m.addConstr(D[r,v]==S[r+1,v], name=f"Dest_{r}_{v}=Src{r+1}_{v}")
                
        #Constraint 3 - A job cannot be picked up before EPT
        for v in range(numAGVs):
            for r in range(REQ[v]):
                m.addConstr(EPT[v][r]<=S[r,v],name=f"S_{r}_{v}>=EPT_{r}_{v}")
                
        #Constraint 4 - To represent lateness Z = D - LDT 
        for v in range(numAGVs):
            for r in range(REQ[v]):
                if LDT[v][r]>0:
                    m.addConstr(Z[r,v]>=D[r,v]+MHT[v][r]- LDT[v][r], name = f"Z_{r}_{v}>=D_{r}_{v}-LDT_{r}_{v}") # where Z[r] represents slack, when Z{r} is -ve, it will be 0 due to lower bound
        
        #Constraint 5 - Battery initialization at starting nodes to 100%
        for v in range(numAGVs):
            m.addConstr(B[0,v]==100, name=f"B_0_init_{v}") #remove hardcoded 100 here

        #Constraint 6 - Battery discharging and charging
        for v in range(numAGVs):
            for r in range(REQ[v]-1):
                i=SRC[v][r]
                j=SRC[v][r+1]
                dist = self.layout.getDistanceFromNode(i,j)
                if i in C and r >=2:
                    b = m.addVar()
                    m.addConstr(b==B[r,v]+((S[r+1,v]-S[r,v]-(1/sp[v])*dist)*cr[v])-((1/sp[v])*dist*dcr[v]), name=f"b_{r}_{v}")
                    m.addGenConstrMin(B[r+1,v],[b,100], name=f"B_{r}_{v}") # charge cannot be greater than 100%
                else:
                    m.addConstr((B[r+1,v]==B[r,v]-((1/sp[v])*dist*dcr[v])), name=f"B_{r}_{v}")
                    
        #Constraint 7 - Check for conflicts
        for v1 in range(numAGVs-1):
            for v2 in range(v1+1,numAGVs): 
                for r1 in range(REQ[v1]-1):
                    for r2 in range(REQ[v2]-1):
                        if SRC[v1][r1]==SRC[v2][r2]:
                            if LDT[v1][r1]<LDT[v2][r2]:
                                dist = self.layout.getDistanceFromNode(source=SRC[v1][r1],dest=SRC[v1][r1+1])
                                m.addConstr(S[r2,v2]>=S[r1+1,v1]-((1/sp[v])*dist)+1,name=f"ConflictS_v{v1}{r1}_v{v2}{r2}")
                            elif LDT[v1][r1]>LDT[v2][r2]:
                                dist = self.layout.getDistanceFromNode(source=SRC[v2][r2],dest=SRC[v2][r2+1])
                                m.addConstr(S[r1,v1]>=S[r2+1,v2]-((1/sp[v])*dist)+1,name=f"Conflict_v{v1}{r1}_v{v2}{r2}")
                        elif DEST[v1][r1]==DEST[v2][r2] and REQID[v1][r1]<998 and REQID[v2][r2]<998:
                            if LDT[v1][r1]<LDT[v2][r2]:
                                dist = self.layout.getDistanceFromNode(source=DEST[v1][r1],dest=DEST[v1][r1+1])
                                m.addConstr(D[r2,v2]>=D[r1+1,v1]-((1/sp[v])*dist)+1,name=f"ConflictD_v{v1}{r1}_v{v2}{r2}")
                            elif LDT[v1][r1]>LDT[v2][r2]:
                                dist = self.layout.getDistanceFromNode(source=DEST[v2][r2],dest=DEST[v2][r2+1])
                                m.addConstr(D[r1,v1]>=D[r2+1,v2]-((1/sp[v])*dist)+1,name=f"Conflict_v{v1}{r1}_v{v2}{r2}")
    
    
        #optimize model
        m.optimize()
        status = m.status
        if status == GRB.Status.UNBOUNDED:
            print('The model cannot be solved because it is unbounded')
        elif status == GRB.Status.OPTIMAL:
            pass
#             print('The optimal objective is %g' % m.objVal)

        elif status == GRB.Status.INF_OR_UNBD or status== GRB.Status.INFEASIBLE:
            print(f'Optimization was stopped with status {status}')
            
#'''
#HEREUNDER LIES THE ALNS IMPLEMENTATION
#'''
    def alns(self):
        
        pass
    
    def extractScheduleKPI(self, taskSchedule):
        '''
        Returns KPI of a given taskSchedule
        '''
        pass
    
    def writeScheduleToFile(self):
        '''
        Writes schedule in a text/excel file and visualizes the schedule
        '''
        pass
    
        

### Execution Flow

In [59]:
from time import *
start=time()
scheduler = Scheduler(layoutFile='outputDM.csv', agvFile='agvs.xlsx', requestFile='transportOrders.xlsx', \
                      stationFile='stations.xlsx')
scheduler.createGreedySequence()
scheduler.createTaskSequence()

scheduler.solveLP(printOutput=False)
end = time()

print(f'time:{end-start}')


TO to schedule:1, type:TO, from:1, to:6, ept:30, ldt:75, cap:B
No. of capable agvs:2
n-agv selected:AGV0,Charge-100%,TCost-1.0
agv info:AGV0,Charge-93.28%,TCost-1.0, release info:(66.8, 6)
task info:1, type:TO, from:1, to:6, ept:30, ldt:75, cap:B
TO to schedule:2, type:TO, from:2, to:7, ept:60, ldt:75, cap:C
No. of capable agvs:1
1-agv selected:AGV1,Charge-100%,TCost-1.5
agv info:AGV1,Charge-91.087717288%,TCost-1.5, release info:(102.28070678, 7)
task info:2, type:TO, from:2, to:7, ept:60, ldt:75, cap:C
TO to schedule:0, type:TO, from:0, to:10, ept:30, ldt:90, cap:A
No. of capable agvs:1
1-agv selected:AGV0,Charge-93.28%,TCost-1.0
agv info:AGV0,Charge-83.43702795733333%,TCost-1.0, release info:(111.40743010666667, 10)
task info:0, type:TO, from:0, to:10, ept:30, ldt:90, cap:A
TO to schedule:3, type:TO, from:4, to:17, ept:90, ldt:120, cap:A
No. of capable agvs:1
1-agv selected:AGV0,Charge-83.43702795733333%,TCost-1.0
agv info:AGV0,Charge-78.13938924666667%,TCost-1.0, release info:(144.6

[2, type:TO, from:2, to:7, ept:60, ldt:75, cap:C,
 4, type:TO, from:5, to:13, ept:105, ldt:150, cap:B,
 5, type:TO, from:8, to:3, ept:120, ldt:280, cap:C,
 8, type:TO, from:1, to:10, ept:300, ldt:447, cap:C,
 11, type:TO, from:5, to:17, ept:400, ldt:481, cap:C,
 14, type:TO, from:8, to:3, ept:491, ldt:554, cap:C,
 17, type:TO, from:1, to:10, ept:590, ldt:661, cap:C,
 20, type:TO, from:5, to:17, ept:689, ldt:839, cap:C]